# Comfy Kettenregel (Autograd DIY) - univariate, skalare Funktionen

$$F(x) = f_1 \circ f_2 = f_1(f_2(x)) \Rightarrow f_1'(f_2(x)) \cdot f'_2(x)$$

$$F(x) = f_1 \circ f_2 \circ f_3 = f_1(f_2(f_3(x))) \Rightarrow f_1'(f_2(f_3(x))) \cdot f_2'(f_3
(x)) \cdot f_3'(x)$$

## Aufgabe

Ziel: Gradientenbasierte Optimierung von $f(x) = \sqrt{\frac{1}{e^{\sin(x)}}}$


### 0. Imports

In [ ]:
import math
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

### 1.0 Operationen definieren

In [ ]:
def one_div_x(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = 1 / x
    derivative = -inner_derivative / x**2

    return value, derivative


def sin(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = math.sin(x)
    derivative = math.cos(x) * inner_derivative

    return value, derivative


def sqrt(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = math.sqrt(x)
    derivative = 1 / (2 * math.sqrt(x)) * inner_derivative

    return value, derivative


def exp(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = math.exp(x)
    derivative = math.exp(x) * inner_derivative

    return value, derivative

### 1.1 Funktionsdefinition

In [ ]:
def f_x(x: float) -> tuple[float, float]:

    return sqrt(*one_div_x(*exp(*sin(x))))

### 2. Gradient Descent

In [ ]:
x_start = 4.0  # starting value
x_min = x_start - 8.0  # x-axis limits
x_max = x_start + 8.0
xs = []  # values for the animation
ys = []

lr = 1e-2  # step size
significant_gradient = 1e-3  # termination criteria
iter = 1  # counter

while True:
    y_measured, deriv = f_x(x_start)
    if np.fabs(deriv) >= significant_gradient:
        xs.append(x_start)
        ys.append(y_measured)
        x_start -= lr * deriv
        print(iter, x_start, y_measured) if iter % 100 == 0 or iter == 1 else None
    else:
        xs.append(x_start)
        ys.append(y_measured)
        break
    iter += 1

### 3.0 Funktionsplot

In [ ]:
x = np.arange(x_min, x_max, 0.01)

res = [f_x(_) for _ in x]
y_measured, derivative = zip(*res)

df = pd.DataFrame(
    {
        "x": x,
        "y": y_measured,
        "derivative": derivative,
    }
)

px.line(df, x="x", y="y")

### 3.1 Animation

In [ ]:
# get the values
x = np.arange(x_min, x_max, 0.01)

res = [f_x(_) for _ in x]
y_measured, _ = zip(*res)

# define both graphs
fig = go.Figure(
    data=[
        go.Scatter(
            x=x,
            y=y_measured,
            mode="lines",
            line=dict(color="green", width=1),
            name="Function Graph",
        ),
        go.Scatter(
            x=[xs[0]],
            y=[ys[0]],
            mode="markers",
            marker=dict(color="red", size=10),
            name="Current Position",
        ),
    ]
)

# update layout parameters and add start button for animation
fig.update_layout(
    width=1400,
    height=900,
    xaxis=dict(range=(x_min, x_max), autorange=False),
    yaxis=dict(
        range=(np.min(y_measured) - 0.5, np.max(y_measured) + 0.5), autorange=False
    ),
    title_text="Gradient Descent Animation",
    # start button config
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(
                    args=[
                        None,
                        {
                            "frame": {"duration": 5, "redraw": False},
                            "fromcurrent": True,
                            "transition": {"duration": 0, "easing": "linear"},
                        },
                    ],
                    label="start",
                    method="animate",
                )
            ],
        )
    ],
)

# specify the animation frames
fig.update(
    frames=[
        go.Frame(data=[go.Scatter(x=[xs[k]], y=[ys[k]])], traces=[1])
        for k in range(len(ys))
    ]
)

# show result
fig.show()

# 2024-11-18 

Bisherige Ansatz hat folgende Limitierungen
- funktioniert nur für Ausdrücke in geschlossener Form, keine Kontrollflusslogik
- inkompatibel mit binären Operatoren (+, *, ...)
- funktioniert nur in 1D

## Value Class

In [ ]:
from __future__ import annotations
import graphviz
from IPython.display import display


class Value:
    def __init__(
        self, value: float, ancestors: tuple[Value, ...] = (), name="", operand=""
    ):
        self.value = value
        self.ancestors = ancestors
        self.name = name
        self.grad = 0.0
        self._backward = lambda: None
        self.operand = operand
        # TODO add separate nodes for operations

    # make values printable
    def __repr__(self) -> str:
        return f"{self.name}, value={self.value}, grad={self.grad}"

    # Addition
    def __add__(self, other: Value) -> Value:
        if not isinstance(other, Value):
            other = Value(other)
        result = Value(self.value + other.value, (self, other), name="add", operand="+")

        def _backward():
            self.grad += 1.0 * result.grad
            other.grad += 1.0 * result.grad

        result._backward = _backward
        return result

    def __iadd__(self, other: Value):
        if not isinstance(other, Value):
            other = Value(other)

        self.value += other.value

        def _backward():
            self.grad += 1.0 * self.grad  # Self gradient should continue accumulating
            other.grad += 1.0 * self.grad

        self._backward = _backward

        return self

    def __radd__(self, other: Value) -> Value:
        if not isinstance(other, Value):
            other = Value(other)
        return self.value + other.value

    # Subtraktion
    def __sub__(self, other: Value) -> Value:
        if not isinstance(other, Value):
            other = Value(other)
        result = Value(self.value - other.value, (self, other), name="sub", operand="-")

        def _backward():
            self.grad += 1.0 * result.grad
            other.grad += -1.0 * result.grad

        result._backward = _backward
        return result

    # Multiplikation
    def __mul__(self, other: Value) -> Value:
        if not isinstance(other, Value):
            other = Value(other)
        result = Value(self.value * other.value, (self, other), name="mul", operand="*")

        def _backward():
            self.grad += other.value * result.grad
            other.grad += self.value * result.grad

        result._backward = _backward
        return result

    # Floatingpointdivision
    def __truediv__(self, other: Value) -> Value:
        if not isinstance(other, Value):
            other = Value(other)
        result = Value(self.value / other.value, (self, other), name="div", operand="/")

        def _backward():
            self.grad += 1 / other.value * result.grad
            other.grad += -self.value / other.value**2 * result.grad

        result._backward = _backward
        return result

    # Potenzierung (x**n)
    def __pow__(self, other: Value) -> Value:
        if not isinstance(other, Value):
            other = Value(other)
        result = Value(self.value**other.value, (self, other), name="pow", operand="^")

        def _backward():
            self.grad += other.value * self.value ** (other.value - 1) * result.grad
            assert self.value >= 0, "cannot compute log with negative bases"
            other.grad += self.value**other.value * np.log(self.value) * result.grad

        result._backward = _backward
        return result

    # backwards up until this point
    # Negation
    def __neg__(self) -> Value:
        result = Value(-self.value, (self,), name="neg", operand="-")

        def _backward():
            self.grad += -result.grad

        result._backward = _backward
        return result

    # Vergleichsoperatoren <, >, >=, <=, ==, !=
    def __lt__(self, other: Value) -> bool:
        return self.value < other.value

    def __gt__(self, other: Value) -> bool:
        return self.value > other.value

    def __le__(self, other: Value) -> bool:
        return self.value <= other.value

    def __ge__(self, other: Value) -> bool:
        return self.value >= other.value

    def __eq__(self, other: Value) -> bool:
        return self.value == other.value

    def __ne__(self, other: Value) -> bool:
        return self.value != other.value

    def __hash__(self) -> int:
        return hash(self.value)

    def backward(self) -> None:
        # iterate through the graph, calculate gradients and update nodes
        topo_sorted_nodes = []
        visited = set()

        # topological sort of the nodes
        def build_topo(node):
            if node not in visited:
                visited.add(node)
                for ancestor in node.ancestors:
                    build_topo(ancestor)
                topo_sorted_nodes.append(node)

        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo_sorted_nodes):
            node._backward()

    def build_graph(self) -> dict:
        def build_node(node) -> dict:
            label = node.__repr__()
            node_info = {
                "node": node,
                "ancestors": [build_node(ancestor) for ancestor in node.ancestors],
                "operand": node.operand,
            }
            return {label: node_info}

        return build_node(self)

    def plot_graph(self):
        # "graph visualization python", graphviz
        dot = graphviz.Digraph(format="svg", graph_attr={"rankdir": "LR"})

        def add_nodes(dot: graphviz.Digraph, node: Value):
            label = f"{node.name}|value={node.value}|grad={node.grad}"
            unique_node_name = str(id(node))

            # add value nodes to graph
            dot.node(
                name=unique_node_name,
                label=label,
                shape="record",
                color="lightgreen" if node.ancestors == () else None,  # check if input
                style="filled",
            )

            if node.operand:  # check if there is an operand to display
                op_name = unique_node_name + node.operand
                # add operation node
                dot.node(
                    name=op_name,
                    label=node.operand,
                )
                # draw edge from operand to result
                dot.edge(op_name, unique_node_name)

            # iterate through the ancestors to build the whole graph
            for ancestor in node.ancestors:
                ancestor_name = add_nodes(dot, ancestor)
                if node.operand:
                    # ensure ancestor edge goes to operand node if it exists
                    dot.edge(ancestor_name, op_name)
                else:
                    dot.edge(ancestor_name, unique_node_name)

            return unique_node_name

        add_nodes(dot, self)
        display(dot)

    # def trace_graph(self) -> tuple[set, set]:
    #     nodes, edges = set(), set()

    #     def build_graph(node):
    #         if node not in nodes:
    #             nodes.add(node)
    #             for ancestor in node.ancestors:
    #                 edges.add((ancestor, node))
    #                 build_graph(ancestor)

    #     build_graph(self)

    #     return nodes, edges

    # def draw_graph(self):
    #     dot = graphviz.Digraph(format="svg", graph_attr={"rankdir": "LR"})

    #     nodes, edges = self.trace_graph()
    #     for n in nodes:
    #         uid = str(id(n))
    #         dot.node(
    #             name=uid,
    #             label=n.__repr__(),
    #             shape="record",
    #             color="lightblue" if n.ancestors == () else None,
    #             style="filled",
    #         )
    #         if n.operand:
    #             dot.node(name=uid + n.operand, label=n.operand)
    #             dot.edge(uid + n.operand, uid)

    #     for n1, n2 in edges:
    #         dot.edge(str(id(n1)), str(id(n2)) + n2.operand)

    #     return dot

In [ ]:
a = Value(2.5, name="a")
b = Value(3.0, name="b")
c = Value(1.5, name="c")


def foo(a: Value, b: Value, c: Value):
    if a > Value(2):
        return a * b + c
    return a - b * c


z1 = foo(a, b, c)
graph = z1.build_graph()
z1.plot_graph()

## Examples

In [ ]:
# initialize values
x = Value(5.0, name="x")
y_measured = Value(2.5, name="y")

a = Value(2.5, name="a")
b = Value(3.0, name="b")
c = Value(1.5, name="c")

# Folgendes sollte ausfühbar sein:
print(x + y_measured)
print(x * y_measured)
print(x - y_measured)
print(x / y_measured)
print(x**y_measured)
print(x**5)
print(-x)
print(x == y_measured)


def foo(a: Value, b: Value, c: Value):
    if a > Value(2):
        return a * b + c
    return a - b * c


def f(a: Value, b: Value, c: Value) -> float:
    # (((b**2) * c) + a)
    x = b**2 * c
    y = a + x
    return y


z1 = foo(a, b, c)
graph = z1.build_graph()
z1.plot_graph()

z2 = foo(Value(-1, name="a2"), b, c)
graph = z2.build_graph()
z2.plot_graph()

z3 = f(a, b, c)
graph = z3.build_graph()
z3.plot_graph()

In [ ]:
z1.backward()
graph = z1.build_graph()
Value.plot_graph(graph)

z3.backward()
graph3 = z3.build_graph()
Value.plot_graph(graph3)

In [ ]:
def funct(x, y):
    return (x + y) ** 2


x = Value(2.5, name="x")
y_measured = Value(5.0, name="y")

vals = funct(x, y_measured)
vals.backward()
Value.plot_graph(vals.build_graph())

# TODO: Lineare Regression

In [ ]:
import numpy as np
import plotly.express as px
from plotly import graph_objects as go

np.random.seed(0xDEADBEEF)

x = np.linspace(-10, 10, 100)
y_ideal = 2 * x - 2
y_measured = y_ideal + np.random.randn(len(x)) * 1.5

fig = px.scatter(x=x, y=y_measured)
fig.add_trace(go.Scatter(x=x, y=y_ideal, mode="lines"))

In [ ]:
# TODO: Lineare Regression f(x) = m*x + c

# Random init von m und c
m = Value(np.random.random_sample(size=None) * 5, name="slope")
c = Value(np.random.random_sample(size=None) * 5, name="intercept")


# Lossfunktion definieren
def loss(m: Value, c: Value) -> Value:
    # sum_error = sum((m * x + c) - y_measured) ** 2
    sum_error = 0.0
    for ii, sample in enumerate(x):
        sample_error = (m * sample + c - y_measured[ii]) ** 2
        sum_error += sample_error
    return sum_error


# Hyperparameter
epochs = 1000
lr = 1e-3
ms = []
cs = []

# Trainingloop
# - Zwischenergebnisse von (m und c) speichern
# - Zwischenergebnisse visualisieren (Animation?)
for i in range(epochs):

    precision_loss = Value(loss(m, c))
    precision_loss.name = "Out"

    m.grad = 0
    c.grad = 0
    precision_loss.backward()

    m.value -= lr * m.grad
    c.value -= lr * c.grad

    if i % 100 == 0:
        print(
            f"epoch {i}: loss = {precision_loss.value, precision_loss.grad, precision_loss.ancestors}, m = {m.value, m.grad}, c = {c.value, c.grad}"
        )
        ms.append(m.value)
        cs.append(c.value)

print(f"final m: {m.value}, final c: {c.value}, final loss: {precision_loss.value}")